# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key



In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,whitehorse,60.7161,-135.0538,48.09,58.0,20.0,8.05,CA,1716823620
1,1,katsuura,35.1333,140.3000,72.19,92.0,100.0,26.17,JP,1716823620
2,2,taiohae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{}
3,3,grez-doiceau,50.7390,4.6983,64.71,75.0,75.0,4.61,BE,1716823621
4,4,grytviken,-54.2811,-36.5092,31.30,82.0,49.0,11.56,GS,1716823621


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
%%capture --no-display
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity"
)
# Display the map
map_plot

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
df_vacation = city_data_df.loc[city_data_df["Max Temp"] > 80]
df_vacation
# Drop any rows with null values
df_vacation = df_vacation.dropna()

# Display sample data
df_vacation.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,rio san juan,19.6439,-70.0763,84.27,75.0,99.0,10.02,DO,1716823622
15,15,katsuren-haebaru,26.3369,127.8719,81.09,89.0,75.0,16.11,JP,1716823623
28,28,weno,7.4515,151.8468,80.55,94.0,100.0,0.00,FM,1716823625
29,29,zouerate,22.7187,-12.4521,93.52,14.0,89.0,11.83,MR,1716823625
31,31,murzuq,25.9155,13.9184,96.39,14.0,100.0,13.89,LY,1716823625


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = df_vacation[["City", "Lat", "Lng", "Country", "Humidity"]].reset_index()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

,index,City,Lat,Lng,Country,Humidity,Hotel Name
0,11,rio san juan,19.6439,-70.0763,DO,75.0,
1,15,katsuren-haebaru,26.3369,127.8719,JP,89.0,
2,28,weno,7.4515,151.8468,FM,94.0,
3,29,zouerate,22.7187,-12.4521,MR,14.0,
4,31,murzuq,25.9155,13.9184,LY,14.0,
...,...,...,...,...,...,...,...
75,246,as sulayyil,20.4607,45.5779,SA,7.0,
76,251,idri,27.5000,13.2667,LY,12.0,
77,252,hope,33.6671,-93.5916,US,55.0,
78,253,camocim,-2.9022,-40.8411,BR,63.0,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
        "apiKey": geoapify_key,
        "format": "json",
        "categories" : categories
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
rio san juan - nearest hotel: Aparta hotel Mi Casa
katsuren-haebaru - nearest hotel: No hotel found
weno - nearest hotel: High Tide Hotel
zouerate - nearest hotel: فندق تيرس زمور
murzuq - nearest hotel: فندق باريس - مرزق
enewetak - nearest hotel: No hotel found
wanning - nearest hotel: No hotel found
uige - nearest hotel: No hotel found
hard bargain - nearest hotel: No hotel found
west island - nearest hotel: Cocos Village Bungalows
puttalam - nearest hotel: Royal Green Garden Hotel
mana - nearest hotel: Le Samana
riyadh - nearest hotel: White Palace Hotel
agadez - nearest hotel: No hotel found
al hada - nearest hotel: No hotel found
tbeng meanchey - nearest hotel: សុភមង្គល
bandarbeyla - nearest hotel: No hotel found
tangalle - nearest hotel: Coco Tangalla
farasan - nearest hotel: No hotel found
labuan - nearest hotel: Labuan Avenue Hotel
bocas del toro - nearest hotel: B&B Surfari
olinda - nearest hotel: Fidji Motel
yanbu - nearest hotel: No hotel found
saipan - 

,index,City,Lat,Lng,Country,Humidity,Hotel Name
0,11,rio san juan,19.6439,-70.0763,DO,75.0,Aparta hotel Mi Casa
1,15,katsuren-haebaru,26.3369,127.8719,JP,89.0,No hotel found
2,28,weno,7.4515,151.8468,FM,94.0,High Tide Hotel
3,29,zouerate,22.7187,-12.4521,MR,14.0,فندق تيرس زمور
4,31,murzuq,25.9155,13.9184,LY,14.0,فندق باريس - مرزق


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    hover_cols=["Hotel Name", "Country"]
    #color = "City"
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country)